In [71]:
from lxml import html 
import requests
from datetime import datetime
from pymongo import MongoClient

In [24]:
from uri import uri

In [25]:
headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

In [79]:
def mail_news():
    total_news = []
    
    headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    
    url = "https://news.mail.ru"
    
    response = requests.get(url, headers=headers)
    dom = html.fromstring(response.text)
    
    first_needed_links = dom.xpath("//div[contains(@class, 'daynews__item')]//@href")
    second_needed_links = dom.xpath("// div[contains(@class, 'daynews js-topnews margin_bottom_30 js-smoky-end')]/following-sibling::ul/li[@class = 'list__item']//@href")

    total_links = first_needed_links + second_needed_links
    
    for link in total_links:
        response = requests.get(link,headers = headers)
        innerdom = html.fromstring(response.text)
    
        source = innerdom.xpath("//span[text()[contains(.,'источник')]]/following-sibling::a//text()")[0]
        news_heading = innerdom.xpath("//h1[contains(@class, 'hdr__inner')]//text()")[0]
        publication_date = innerdom.xpath("//span[contains(@class,'note')]//span[@datetime]/@datetime")[0]
    
        news = {
        'title': news_heading,
        'source': source,
        'publication_date':publication_date,
        'link':link
        }
    
    
        total_news.append(news)
        
    uri = uri
    client = MongoClient(uri,27017)
    db = client["users2907"]
    db.news.insert_many(total_news)
    
   
    
    

In [81]:
mail_news()